This notebook relies on the following cmd line tools, ensure they are installed and in the system
- tippecanoe
- mapshaper
- aws cli

In [1]:
from pathlib import Path
import sys
import logging

from IPython.lib import backgroundjobs as bg

scripts_dir = Path('..').joinpath('src')
if scripts_dir not in sys.path:
    sys.path.insert(0, scripts_dir.resolve().as_posix())

from pipelines.pipes import get_pipes, execution_order, filter_pipes

logging.basicConfig(level=logging.DEBUG)
logging.getLogger("requests").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)
logging.getLogger("fiona").setLevel(logging.WARNING)
jobs = bg.BackgroundJobManager()

In [2]:
# def execute_pipes(pipes):
#     ordered_pipes = execution_order(pipes)
#     for pipe in ordered_pipes:
#         jobs.new(pipe().execute)

# # this code will execute all pipelines selected in background.

# mypipes_subset = filter_pipes(get_pipes().items(), ['eez_tiles'])
# execute_pipes(mypipes_subset)

In [3]:
get_pipes()

{'eez_intermediate': pipelines.intermediate_pipes.EEZIntermediatePipe.EEZIntermediatePipe,
 'eez_tiles': pipelines.tiles_pipes.EEZTilesPipe.EEZTilesPipe,
 'eez_precalc': pipelines.precalc_pipes.EEZPrecalcPipe.EEZPrecalcPipe}

In [4]:
mypipes_subset = filter_pipes(get_pipes(), ['eez_intermediate'])

In [5]:
for n, pipe in mypipes_subset.items():
    new_pipe = pipe()
    new_pipe.extract().transform().load()

DEBUG:pipelines.settings:/home/mambauser/data
INFO:pipelines.base_pipe:Pipeline eez_intermediate running at 2023-11-02 10:12:39.302930: starting extract...
INFO:utils:File /home/mambauser/data/eez_intermediate/World_EEZ_v11_20191118.zip already exists.
INFO:utils:File /home/mambauser/data/eez_intermediate/World_High_Seas_v1_20200826.zip already exists.
INFO:pipelines.base_pipe:Pipeline eez_intermediate finish at 2023-11-02 10:12:39.304361: Success executing extract
INFO:pipelines.base_pipe:Pipeline eez_intermediate running at 2023-11-02 10:12:39.304695: starting transform...
INFO:pipelines.base_pipe:Pipeline eez_intermediate finish at 2023-11-02 10:12:39.305303: Success executing transform
INFO:pipelines.base_pipe:Pipeline eez_intermediate running at 2023-11-02 10:12:39.305575: starting load...
DEBUG:google.auth.transport.requests:Making request: POST https://oauth2.googleapis.com/token
ERROR:pipelines.base_pipe:Pipeline eez_intermediate dead at 2023-11-02 10:12:39.608183: Traceback (m